<a href="https://colab.research.google.com/github/Victor0vich/Denis/blob/main/Cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Код мастер-класса:
import telebot
from telebot import types
import json
import time
import requests
import base64

api_key = ""
secret_key = ""
token = ""

bot = telebot.TeleBot(token)

styles = {
    'KANDINSKY': 'KANDINSKY',
    'Детальное фото': 'UHD',
    'ANIME': 'ANIME',
    'No style': 'DEFAULT'
}


class GenerateModel:  # Создаем класс для работы с API
    def __init__(self, url, api_key, secret_key):
        self.URL = url
        self.AUTH_HEADERS = {
            'X-Key': f'Key {api_key}',
            'X-Secret': f'Secret {secret_key}',
        }

    def get_model(self):
        response = requests.get(self.URL + 'key/api/v1/models', headers=self.AUTH_HEADERS)
        data = response.json()
        return data[0]['id']  # id модели

    def generate(self, prompt, model, style, images=1, width=1024, height=1024):
        params = {
            "type": "GENERATE",
            "numImages": images,
            "width": width,
            "height": height,
            "style": style,
            "generateParams": {
                "query": f"{prompt}"
            }
        }
        data = {
            'model_id': (None, model),
            'params': (None, json.dumps(params), 'application/json')
        }
        response = requests.post(self.URL + 'key/api/v1/text2image/run', headers=self.AUTH_HEADERS, files=data)
        data = response.json()
        return data['uuid']  # уникальный идентификатор задачи на генерацию

    def check_generation(self, request_id, attempts=10, delay=10):
        while attempts > 0:
            response = requests.get(self.URL + 'key/api/v1/text2image/status/' + request_id, headers=self.AUTH_HEADERS)
            data = response.json()
            if data['status'] == 'DONE':
                return data['images']
            attempts -= 1
            time.sleep(delay)


@bot.message_handler(commands=['start'])
def welcome_func(message):
    bot.send_message(message.chat.id,
                     "Привет! Я бот, который генерирует картинки по твоему запросу! Чтобы начать, отправь мне команду /generate")


@bot.message_handler(commands=['generate'])
def generate_func(message):
    kb = types.ReplyKeyboardMarkup()
    for key in styles.keys():
        button = types.KeyboardButton(key)
        kb.add(button)
    bot.send_message(message.chat.id, "Выбери стиль:", reply_markup=kb)


@bot.message_handler(func=lambda message: message.text in list(styles.keys()))  # ???
def generate_with_style(message):
    bot.send_message(message.chat.id, f'Вы выбрали стиль: {message.text}.Введите свой промт')
    style = styles[message.text]
    bot.register_next_step_handler(message, generate_picture, style)


def generate_picture(message, style):
    promt = message.text
    url = 'https://api-key.fusionbrain.ai/'
    api = GenerateModel(url, api_key=api_key, secret_key=secret_key)
    model_id = api.get_model()
    uuid = api.generate(promt, model_id, style)
    images = api.check_generation(uuid)
    if images != None:
        images_base64 = images[0]
        image_data = base64.b64decode(images_base64)
        with open('generate_images.jpg', 'wb') as f:
            f.write(image_data)
        with open('generate_images.jpg', 'rb') as f:
            bot.send_photo(message.chat.id, f, caption=promt)
    else:
        bot.send_message(message.chat.id, 'Не удалось сгенерировать изображение')


bot.infinity_polling()

SyntaxError: invalid syntax (<ipython-input-1-063edcc2d7da>, line 1)